[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/prokaj/elte-python/blob/main/10-gyakorlat.ipynb)

# Házi feladatok

## 1. Maximum eloszlása szimmetrikus bolyongásnál

Szimmetrikus bolyongás alatt olyan véletlen sorozatot értünk, ami a nullából indul és minden lépése (megváltozása 1 vagy −1, 1/2–1/2 valószínűséggel a többi lépéstől függetlenül.

Írjunk egy függvényt, ami a numpy könyvtár függvényei segítségével elkészíti n hosszú bolyongásoknak egy m elemű sorozatát! Azaz a függvény kimenete egy $m\times n$ mátrix, az egyes sorok az egyes bolyongásokat írják le.

Rajzoljunk ki néhány bolyongást! Készítsünk hisztogramot a bolyongások utolsó pozíciójából (ez a függvény által visszakapott mátrix utolsó oszlopa). Valamint készítsünk hisztogramot a bolyongások maximális értékéből is (ezek a mátrixunk soronkénti maximumai). $n$ legyen legalább 100 és $m$ legalább 1000. Észreveszünk-e valami összefüggést az ábrák között?

A megoldás során jól jöhetnek a következő függvények:

```
import numpy as np
import matplotlib.pyplot as plt
np.random.binomial, np.cumsum, np.where, np.max
plt.plot, plt.hist, plt.title, plt.grid
```

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

def random_walk(n, m=1):
    steps = np.empty((m, n+1), dtype=np.int32)
    steps[:, 1:] = 2*np.random.binomial(1, 0.5, size=(m,n))-1
    steps[:, 0] = 0
    return steps.cumsum(axis=1)



In [ ]:
import ipytest
ipytest.autoconfig()

In [ ]:
%%ipytest

def test_random_walk():
    rw = random_walk(10)
    assert rw.shape == (1,11)

    rw = random_walk(100)
    assert set(np.diff(rw[0])) == {-1, 1}

    rw = random_walk(100, 1000)
    assert (rw[:, 0]==0).all()
    
    

In [ ]:
rw = random_walk(10000, 4)
for i in range(4):
    plt.plot(rw[i], linewidth=0.5)
plt.grid()
plt.show()

In [ ]:
rw = random_walk(100, 50000)

In [ ]:
rw_last  =  rw[:, -1]
plt.hist(rw_last, density=True)
plt.grid()
plt.show()

bins = np.arange(rw_last.min()-1, rw_last.max()+2, 2)
plt.hist(rw_last, bins, density=True)
plt.grid()
plt.show()

In [ ]:
rw_max = rw.max(axis=1)
print(rw_max.shape)

plt.hist(rw_max, density=True)
plt.grid()
plt.show()

bins = np.arange(rw_max.max()+2)-0.5
plt.hist(rw_max, bins, density=True)
plt.grid()
plt.show()

## Magyarázat

A két ábra nagyon hasonlít, a maximumok hisztogrammja olyan mint felső ábra pozitív fele.

A szimmetikus bolyongás egy véletlen sorozat, a szokásos jelölés $(S_k)_{0\leq k\leq n}$.

A maximum eloszlása pl. tükrözési elvvel számolható. $\ell\geq0$-ra:
$$
\mathbb{P}(\max_{k\leq n} S_k\geq x) = 
\mathbb{P}(\max_{k\leq n} S_k\geq x,\, S_n\geq x) +
\mathbb{P}(\max_{k\leq n} S_k\geq x,\, S_n<x)
$$
A második taggal kell foglalkozni. Nézzük azt az esetet, amikor $S_n=y< x$.

Minden lehetséges út egyformán valószínű ($1/2^n$). Így az  a kérdés, hány olyan út van ami $(0,0)$-ból $(n,y)$-ba, aminek maximuma legalább $x$. Az ilyen utak a *jó* utak.

Egy jó út esetén legyen $t$ az első olyan index, amikor az $x$ szinten jár, és a $t$ utáni lépéseket fordítsuk az ellentettjükre.

In [ ]:

x = 8
idx = np.where((rw_last<x) & (rw_max>=x))[0]

if len(idx)>0:
    idx = idx.min()
    rw0 = rw[idx]
    t = np.where(rw0==x)[0].min()
    rw1 = np.zeros_like(rw0)
    rw1[:t+1] = rw0[:t+1]
    rw1[t+1:] = 2*rw0[t]-rw0[t+1:]
    plt.plot(rw0, label="eredeti út", alpha=0.5)
    plt.plot(rw1, label="tükrözött út", alpha=0.5)
    plt.hlines([ell], 0, len(rw0))
    plt.plot(t, ell, 'ro')
    plt.grid()
    plt.legend()


Ha $x\geq 0$ és $y<x$, akkor 
a fenti hozzárendelés bijekció azon utak között, amik $(0,0)$-ból $(n,y)$-ba mennek és közben járnak $x$ magasságban és azon utak között, amik 
$(0,0)$-ból $(n,2x-y)$ mennek. A tükrözött útból úgy kaphatjuk vissza az eredetit, hogy $t$ a tükrözött útból is megkapható, mint az első olyan index, amikor az $x$ szinten jár a bolyongás, és a $t$ utáni növekményeket $-1$-gyel szorozzuk.
$$
\begin{aligned}
\mathbb{P}(\max_{k\leq n} S_k\geq x) &= 
\mathbb{P}(\max_{k\leq n} S_k\geq x,\, S_n\geq x) +
\mathbb{P}(\max_{k\leq n} S_k\geq x,\, S_n<x)\\
&= \mathbb{P}(S_n\geq x) +
\mathbb{P}(S_n>x)
\end{aligned}
$$

Ebből
$$
\mathbb{P}(\max_{k\leq n} S_k = x) = \mathbb{P}(S_n = x) + \mathbb{P}(S_n = x+1)
$$

A `plt.hist` függvény használata helyett célszerű magunknak kigyűjteni a gyakoriságokat.

In [ ]:
rw_last_values, rw_last_cnts = np.unique(rw_last, return_counts=True)
rw_last_probs = rw_last_cnts/rw_last_cnts.sum()

rw_max_values, rw_max_cnts = np.unique(rw_max, return_counts=True)
rw_max_probs = rw_max_cnts/rw_max_cnts.sum()

plt.bar(rw_last_values, rw_last_probs, width=2, alpha=0.5)
plt.bar(rw_max_values, rw_max_probs, width=1, alpha=0.5)
plt.show()

Be tudjuk-e rajzolni a haranggörbét?

Ehhez egyrészt használhatjuk a `scipy.stats` `normal` objektumának `pdf` (probablity density function) metódusát. Vagy megírhatjuk magunk is.

Vegyük észre, hogy 100 lépés után az átlag 0  a szórás $10=\sqrt{100}$. Pl. az mintában amit generáltunk ezek az értékek:

In [ ]:
print(f"minta átlag = {rw_last.mean():.3f}, minta szórása={rw_last.std():.3f}")

In [ ]:
from scipy.stats import norm as gauss

In [ ]:
sigma = (rw.shape[1]-1)**0.5
plt.plot(rw_last_values, gauss.pdf(rw_last_values, 0, sigma))
plt.grid()
plt.show()

In [ ]:
plt.bar(rw_last_values, rw_last_probs, width=2, alpha=0.5)
plt.bar(rw_max_values, rw_max_probs, width=1, alpha=0.5)
plt.plot(rw_last_values, 2*gauss.pdf(rw_last_values, 0, sigma))
plt.show()

## 2. Ugyanez minimummal. 

## 3. Utolsó nullhely.

Szimmetrikus bolyongás alatt olyan véletlen sorozatot értünk, ami a nullából indul és minden lépése (megváltozása) 1 vagy −1, 1/2–1/2
valószínűséggel a többi lépéstől függetlenül.

Írjunk egy függvényt, ami a numpy könyvtár függvényei segítségével elkészíti $n$
hosszú bolyongásoknak egy $m$ elemű sorozatát! Azaz a függvény kimenete egy $m\times n$
mátrix, az egyes sorok az egyes bolyongásokat írják le.

Rajzoljunk ki néhány bolyongást!

A következő részhez $n$ legyen legalább 100 és $m$ legalább 1000.

Minden bolyongásra számoljuk ki az utolsó nulla érték indexét és ezt osszuk el a bolyongás hosszával. Készítsünk hisztogramot az így kapott értékekből.

Extra kérdés, a pontozásba nem számít bele: Van-e olyan szép függvény, ami illeszkedik a hisztogram alakjára.

A bolyongás legyártásában nincs különbség. Az utolsó nullhelyet nehézkesebb csak array műveletekkel számolni.

In [ ]:
def last_zero(rw):
    return np.array([np.where(row==0)[0].max() for row in rw])

def last_zero_2(rw):
    return ((rw==0)*np.arange(0, rw.shape[1])).max(axis=1)


In [ ]:
%%ipytest

def last_zero_row(row):
    idx = 0
    for i, x in enumerate(row):
        if x==0:
            idx = i
    return idx

def last_zero_with_for(rw):
    return np.array([last_zero_row(row) for row in rw])
    

def test_last_zero():
    rw = random_walk(100,1000)
    assert (last_zero(rw) == last_zero_with_for(rw)).all()
    assert (last_zero_2(rw) == last_zero_with_for(rw)).all()
    

In [ ]:

rw_last_zero = last_zero(rw)/(rw.shape[1]-1)
print(f"{rw_last_zero.min()=}, {rw_last_zero.max()=}")
plt.hist(rw_last_zero)


In [ ]:
rw_last_zero_values, rw_last_zero_cnts = np.unique(rw_last_zero, return_counts=True)
rw_last_zero_probs = rw_last_zero_cnts/rw_last_zero_cnts.sum()
width = np.diff(rw_last_zero_values).max()
plt.bar(rw_last_zero_values, rw_last_zero_probs, width=0.9*width)
plt.show()

## Magyarázat.

A fenti számolásból
$$
    \mathbb{P}(S_{1}\geq 0, \dots S_n\geq 0) = \mathbb{P}(\min_{k\leq n} S_k\geq 0) = \mathbb{P}(\max_{k\leq n} S_k\leq 0) = \mathbb{P}(\max_{k\leq n} S_k= 0) = \mathbb{P}(S_n=0)+\mathbb{P}(S_n=1) = \mathbb{P}(S_n=0)+\mathbb{P}(S_{n+1}=0) 
$$

Innen 
$$
    \mathbb{P}(S_{1}> 0, \dots S_n> 0) = \frac12\mathbb{P}(\min_{k\leq n} S_k-S_1\geq 0) = \frac12\left(\mathbb{P}(S_{n-1}=0)+\mathbb{P}(S_{n}=0)\right)
$$
és 
$$
    \mathbb{P}(S_{1}\neq 0, \dots S_n\neq 0) = 
    \mathbb{P}(S_{1}> 0, \dots S_n> 0)+\mathbb{P}(S_{1}< 0, \dots S_n< 0) = \mathbb{P}(S_{n-1}=0)+\mathbb{P}(S_{n}=0)
$$

Legyen $n=2k$ páros és $T_n$ az utolsó nullhely az első $n$ lépés során
$$
    \mathbb{P}(T_n = 2\ell)=
    \mathbb{P}(S_{2\ell}=0,\, S_{2\ell+i}-S_{2\ell}\neq 0,\, i=1,\dots,2k-2\ell)
    =\mathbb{P}(S_{2\ell}=0)\mathbb{P}(S_{2(n-\ell)}=0)
    =\frac{1}{2^{2\ell}}\binom{2\ell}{\ell}\frac{1}{2^{2(k-\ell)}}\binom{2(k-\ell)}{k-\ell}
$$

HF középiskolából: rekurzió $I_k = \int_0^{\pi/2}\cos^k(\alpha)d\alpha$. Ha ezt felírjuk, kihasználjuk, hogy $I_0=\pi/2$, $I_1=1$, akkor azt kapjuk, hogy
$$
    \frac1{\sqrt{\pi(k+1/2)}}\leq \frac{1}{2^{2k}}\binom{2k}{k}\leq \frac{1}{\sqrt{\pi k}},
    \quad
    \frac1{\sqrt{\pi(k+1/4)}}\approx \frac{1}{2^{2k}}\binom{2k}{k}
$$

Azaz 

$$
    \mathbb{P}(T_n=2\ell) \approx \frac{2}{n}\left.\frac{1}{\pi}\cdot\frac1{\sqrt{x(1-x)}}\right|_{x=\frac{2\ell}{n}}
$$

,,Ellenőrizzük'' a kapott formulát 

In [ ]:
def arcsin_probs(x):
    width = np.diff(rw_last_zero_values).max()
    d = np.zeros_like(x)
    y = (x+0.33*width)*(1-x+0.33*width)
    d[y>0] = width/(np.sqrt(y[y>0])*np.pi)
    return d
    
plt.bar(rw_last_zero_values, rw_last_zero_probs, width=0.9*width)
plt.plot(rw_last_zero_values, arcsin_probs(rw_last_zero_values), color='red')
plt.show()

# `with` statement


 
## Mi történik a `with` statement alkalmazásakor?

### Mit lehet a with mögé írni?

Hasonlóan a `for`-hoz szinte bármit, aminek van két metódusa:

- `__enter__`
- `__exit__`


    with obj as x:
       do_something with x

Itt az `obj.__enter__()` hívás eredménye lesz az x értéke és a block végén **GARANTÁLTAN** végrehajtódik az `obj.__exit__(...)` hívás.
Az `__enter__` metódus végezheti az előkészítést, az `__exit__` a takarítást!

#### Kell-e nekünk ezeket a metódusokat közvetlenül implementálni?

Valójában nem. Elegendő egy generátor függvényt megírni:

    import time
    
    def timer():
        try:
            start = time.time()
            yield

        finally:
            runtime = time.time() - start
            print(f"run time: {runtime:3f}")
            

Így még csak egy generátort kapunk ami egyszer visszaad semmit (`None`) majd jelzi, hogy vége van a sorozatnak. 

Próbáljuk ki `for`-ral

In [ ]:
import time
    
def timer():
    try:
        start = time.time()
        yield

    finally:
        runtime = time.time() - start
        print(f"run time: {runtime:3f}")
        
for x in timer():
    print(x)

`with`-del hibát kapunk

In [ ]:
with timer() as x:
    print(x)

A `contextlib` module `contextmanager` függvénye generátorból `contextmanager`-t készít.

In [ ]:
from contextlib import contextmanager
timer2 = contextmanager(timer)
with timer2() as x:
    print(x)

Ha egy függvény akarunk alkalmazni egy függvényre, hogy azt átalakítsuk, de ugyanaz maradjon a neve akkor a python `@` szintakszist használja. Ilyenkor ,,dekoráljuk'' a függvényt. A `contextmanager` egy példa **dekorátor**ra.

    @contextmanager
    def timer():
        try:
            start = time.time()
            yield

        finally:
            runtime = time.time() - start
            print(f"run time: {runtime:3f}")
    
Példaként írjunk egy olyan contextmanager-t, ami jelzi nekünk, hogy mikor milyen hívás történik.

In [ ]:
@contextmanager
def print_whats_going_on(x):
    print("try blokk előtt")
    try:
        print("yield előtt")
        yield x
        print("yield után")
    except:
        print("except ág")
    finally:
        print("finally ág")
    print("try blokk után")

In [ ]:
with print_whats_going_on("hello") as x:
    print(x)
print("with után")

print("-"*50)

with print_whats_going_on("hello") as x:
    print(x)
    raise ValueError
print("with után")


In [ ]:
@contextmanager
def timer():
    try:
        start = time.time()
        yield

    finally:
        runtime = time.time() - start
        print(f"run time: {runtime:3f}")

In [ ]:
t = timer()
with t:
    time.sleep(0.5)

Használhatjuk-e többször a `timer`-ünket?

In [ ]:
t = timer()
with t:
    time.sleep(0.5)
with t:
    time.sleep(0.5)


## Tudunk-e olyan contextmanagert készíteni, ami többször felhasználható?

pl.

```
with indented:
    indented.print("első")
    with indented:
        indented.print("második")
    indented.print("harmadik")
```

ahol kimenetként azt várnánk, hogy
```
  első
    második
  harmadik
```

In [ ]:
class Indent:
    def __init__(self, indent='  '):
        self.level = 0
        self.indent = indent
    
    def __enter__(self):
        self.level += 1
        return None
    
    def __exit__(self, *args):
        self.level -= 1
    
    def print(self, x):
        indent = self.indent*self.level 
        x = str(x)
        print('\n'.join(indent+line for line in x.split('\n')))


In [ ]:
indented = Indent('...')
text = """Többsoros szöveg:
Első
Második
"""
with indented:
    indented.print("első")
    with indented:
        indented.print("második")
        with indented:
            indented.print(text)
    indented.print("harmadik")

# Feladatok előadásról

Előadáson a föld népességének alakulása szerepelt egy [Kaggle](https://www.kaggle.com) adathalmazon. Mi a 
[https://www.gapminder.org/data/] oldalról származó adatokat fogjuk használni. 

FREE DATA FROM WORLD BANK VIA GAPMINDER.ORG, CC-BY LICENSE

Felmásoltam őket az óra github repojára, a letöltési címek:


In [ ]:
csv_files = {
    "income": "income_per_person_gdppercapita_ppp_inflation_adjusted.csv",
    "life_expectancy": "life_expectancy_years.csv",
    "population": "population_total.csv"
}
regions_url = "http://gapm.io/dl_geo"

baseurl = "https://github.com/prokaj/elte-python/raw/main/"

## Kérdések előadásról:

* Melyek a Föld legsűrűbben lakott országai?
* Melyek a legritkábban lakott országok?
* Földrészenként melyik a három legritkábban lakott ország?
* Melyik földrész népessége nőtt a legtöbbet 1970 óta?

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:

gapminder = px.data.gapminder()
gapminder


In [ ]:
print(gapminder.describe())
print(gapminder.info())


In [ ]:
px.scatter(data_frame= gapminder,
           x= "gdpPercap",
           y="lifeExp", 
           animation_frame="year",
           animation_group="country",
           size="pop", 
           color="continent", 
           hover_name="country",
           log_x=True, 
           size_max=55, 
           range_x=[100, 100000], 
           range_y=[25,90])

Részletesebb adatok:

In [ ]:

url = baseurl+csv_files['income']
print(url)
df = pd.read_csv(url, dtype=str)
df.head()

In [ ]:
from collections import defaultdict
col_types = defaultdict(list)
for col in df.columns:
    col_types[str(df[[col]].values.dtype)].append(col)

print(dict(col_types))
    


## Feladat.

Olvassuk be az adatokat és válaszoljuk meg a fenti kérdéseket: 

* Melyek a Föld legsűrűbben lakott országai?
* Melyek a legritkábban lakott országok?
* Földrészenként melyik a három legritkábban lakott ország?
* Melyik földrész népessége nőtt a legtöbbet 1970 óta?

# Conway Game of Life


The Game of Life is a cellular automaton created by mathematician John Conway in 1970. The game consists of a board of cells that are either on or off. One creates an initial configuration of these on/off states and observes how it evolves. There are four simple rules to determine the next state of the game board, given the current state:

- **Overpopulation**: if a living cell is surrounded by more than three living cells, it dies.
- **Stasis**: if a living cell is surrounded by two or three living cells, it survives.
- **Underpopulation**: if a living cell is surrounded by fewer than two living cells, it dies.
- **Reproduction**: if a dead cell is surrounded by exactly three cells, it becomes a live cell.


Írjunk egy osztályt a játékhoz, pl. az `__init__` metódus hozzalétre a megadott konfigurációnak megfelelő objektumot. Legyen egy `step` metódus, ami a rendszert a következő állapotába viszi és az `__str__` metódus pedig
valahogy ábrázolja az aktuális állapotot. 

Tegyük fel, hogy a rács amin a rendszer él, egy $n\times n$-es rács, ahol mindkét irányban ciklikusan körbemegyünk,
azaz a csúcsokat modulo $n$ tekintjük.


In [ ]:
class ConwayGoL:
    
    def __init__(self, state):
        self.state = list(state)

    def step(self):
        return self

    def __repr__(self):
        return f"{type(self).__name__}({self.state})"

In [ ]:
import random

init_state = [] ## ???
conway = ConwayGoL(init_state)

conway.step()


A játék állapotának leírásához egy $n\times n$ rács minden pontjáról tudni kell, hogy foglalt-e vagy sem.

```
n = 11
state = [[0]*n for _ in range(n)]
```

Véletlenszerű kezdeti állapot:
```
state = [[random.randint(0,1) for _ in range(n)] for _ in range(n)]
```

In [ ]:
def random_state(n, m, p):
    return [[int(random.random()<p) for _ in range(m)] for _ in range(n)]

state = random_state(11, 25, 0.2)
print(state)

Szebb megjelenítés?

In [ ]:
def as_matrix(lst, n):
    return [lst[i:i+n] for i in range(0, len(lst), n)]

print('\n'.join(''.join(map(str, line)) for line in state))


In [ ]:
symbols = "\u2b1c\u2b1b"
print('\n'.join(''.join(symbols[x] for x in line) for line in state))


In [ ]:
import matplotlib.pyplot as plt 

img = plt.matshow(state, cmap="viridis")
#img.axes.axis(False)
n, m = len(state), len(state[0])
for pos in range(0, n+1):
    img.axes.axhline(y=pos-0.5, color="white")
for pos in range(0, m+1):
    img.axes.axvline(x=pos-0.5, color="white")

plt.show()


In [ ]:

def cgol_str(self):
    symbols = "\u2b1c\u2b1b"
    return '\n'.join(''.join(symbols[x] for x in line) for line in self.state)

ConwayGoL.__str__ = cgol_str

    

In [ ]:
conway = ConwayGoL(state)
print(conway)

A `step` metódushoz ki kellene számolni egy adott csúcs foglalt szomszédainak számát `cnt`. Ha ez kész,
akkor az $i$ csúcs új állapota:

$$
    \text{state}_{t+1}[i]=
    \begin{cases}
    1 &\text{Ha $\text{cnt}[i]\in\{2,3\}$ és $\text{state}_t[i]=1$}\\
    1 &\text{Ha $\text{cnt}[i]\in\{3\}$ és $\text{state}_t[i]=0$}\\
    0 &\text{különben}
    \end{cases}
$$

In [ ]:
def newstate(state, count):
    return [int((c==3)|((c==2) & (s==1))) for s, c in  zip(state, count)]

In [ ]:
%%ipytest

def test_newstate():
    res = [0]*9
    res[2] = 1
    res[3] = 1
    assert newstate([1]*9, list(range(9))) == res
    res = [0]*9
    res[3] = 1    
    assert newstate([0]*9, list(range(9))) == res


In [ ]:
def count_neighbors(state):
    strides = [(0,-1), (0, 1), (1,-1), (1,0), (1,1), (-1,-1), (-1,0), (-1,1)] 
    n, m = len(state), len(state[0])
    return [ [sum(state[(i+di) % n][(j+dj) % m] for di, dj in strides) for j in range(m)] for i in range(n)]

def cgol_step(self):
    counts = count_neighbors(self.state)
    self.state = [ newstate(line, cnt) for line, cnt in zip(self.state, counts) ]
    return self
    
ConwayGoL.step = cgol_step

In [ ]:
@classmethod
def cgol_from_random_state(cls, n, m,  p):
    return cls(random_state(n, m, p))

ConwayGoL.from_random_state=cgol_from_random_state

In [ ]:
conway = ConwayGoL.from_random_state(4, 4, 0.3)
print(conway)
print(count_neighbors(conway.state))

In [ ]:
conway = ConwayGoL.from_random_state(5, 5, 0.5)
print(conway)
print("-"*20)
print(conway.step())

Tudunk-e valami animációszerűt készíteni? Jupyter notebook-ban pl. a következő képpen lehet: 

In [ ]:
from ipywidgets import Output
from time import sleep

out = Output()
display(out)
conway = ConwayGoL.from_random_state(n=31, m=55, p=0.2)

for i in range(100):
    out.clear_output(True)
    with out:
        print(f"After {i} steps:\n{conway}")  
    sleep(0.2)
    conway.step()


Ha parancssorból dolgozunk, akkor valami ilyesmit lehetne tenni

In [ ]:
%%writefile conway.py

import random


def random_state(n, m, p):
    return [ [ int(random.random()<p) for _ in range(m) ] for _ in range(n) ]

def count_neighbors(state):
    strides = [(0,-1), (0, 1), (1,-1), (1,0), (1,1), (-1,-1), (-1,0), (-1,1)] 
    n, m = len(state), len(state[0])
    return [ [sum(state[(i+di) % n][(j+dj) % m] for di, dj in strides) for j in range(m)] for i in range(n)]

def newstate(state, count):
    return [int((c==3)|((c==2) & (s==1))) for s, c in  zip(state, count)]

class ConwayGoL:
    symbols = "\u2b1c\u2b1b"
    
    def __init__(self, state):
        self.state = list(state)
        
    def step(self):
        counts = count_neighbors(self.state)
        self.state = [ newstate(line, cnt) for line, cnt in zip(self.state, counts) ]
        return self

    
    def __str__(self): 
        symbols = self.symbols
        return '\n'.join(''.join(symbols[x] for x in line) for line in self.state)

    def __repr__(self):
        return f"{type(self).__name__}({self.state})"
    
    @classmethod
    def from_random_state(cls, n, m, p):
        return cls(random_state(n, m, p))

    def is_empty(self):
        return not any(any(line) for line in self.state)


def clear_terminal(n):
    print(f"{chr(27)}[{n+1}A", end="")

def main(n=11, m=25, p=0.2, nsteps=10, clear_screen=clear_terminal):
    from time import sleep
    conway = ConwayGoL.from_random_state(n, m, p)
    for i in range(nsteps+1):
        if i>0:
            clear_screen(n)
        print(f"after {i} step:")
        print(conway)  
        sleep(0.2)
        conway.step()
        if conway.is_empty():
            break
    
if __name__ == "__main__":
    main()
    

Ha valamit már megírtunk és szeretnénk használni, `import`-tal elérhető. Pl.

### Tudunk-e paramétereket adni a python scriptnek?

Amikor egy python scriptet futtatunk, a parancssor (amivel a futást indítottuk) a `sys` modul `argv` változójában érhető el.

In [ ]:
import sys
sys.argv

In [ ]:
! python -c 'import sys; print(sys.argv)' -alma


Egy nagyon egyszerű megoldás, ha minden opciónak a neve a paraméter amit beállít és egyenlőségjel után az értéke:
pl. n=11 m=25 nstep=10 p=0.2

In [ ]:
cmdline = "conway.py -n=11 -m=25 -nstep=10 -p=0.2"
argv = cmdline.split()
params =[param.split("=") for param in argv[1:]]
params

Minden paraméterről tudni kellene, hogy milyen típusú!

In [ ]:
param_types={'-n': int, '-m': int, '-nstep': int, '-p': float}
params = {k.replace("-",""): param_types[k](v)  for k, v in (param.split("=") for param in argv[1:])}
params

Ezek után a `main` függvényt a megadott paraméterekkel meg tudjuk hívni:

```
    main(**params)
```
Mi van a `default` értékekkel, `help`-pel stb.

Ezeket mind meg tudnánk írni, de nem kell. Van kész megoldás `python`-ban.

Az `argparse` könvytár mindent megcsinál, ami nekünk kell.

In [ ]:
import argparse

help(argparse)

A `conway.py` file végét cseréljük le erre.
```
if __name__=="__main__":

    import argparse
    
    parser = argparse.ArgumentParser(description='Conways Game of Life')

    parser.add_argument(
        '-n', '--nrows', 
        type=int, 
        default=11, 
        help='number of rows'
        )

    parser.add_argument(
        '-m', '--ncols',
        type=int, 
        default=25, 
        help='number of columns'
        )

    parser.add_argument(
        '-p', '--density',
        type=float, 
        default=0.2,
        help='initial density')
    
    parser.add_argument(
        '--nsteps', 
        type=int, 
        default=10, 
        help='steps to display'
        )

    args = parser.parse_args()
    
    main(n=args.n, m=args.m, p=args.p, nsteps=args.nsteps)
```
    

In [ ]:
# import importlib
# importlib.reload(conway)

In [ ]:
import conway

out1 = Output()
display(out1)

with out1:
    conway.main(clear_screen=lambda n: out1.clear_output(True))

Az `argparse` könyvtár nem a legkényelmesebb. Alternatívák:

- [Docopt](http://docopt.org/)
- [Click](https://pypi.org/project/click/)
- [clize](https://github.com/epsy/clize)

és még sok másik is!

## Itt is használhattunk volna dekorátort


A `ConwayGoL` példában utólag adtunk metódusokat az osztályunkhoz. Ezt is megtehettük volna dekorátorral.  

In [ ]:
def conway_method(f):
    setattr(ConwayGoL, f.__name__, f)
    return f


@conway_method
def dummy_method(self):
    print("this is a message from the new method!")

c = ConwayGoL([], 0)
c.dummy_method()

Azt is megtehettük volna, hogy a osztály nincs beleégetve a kódba.

In [ ]:
def new_method(cls):
    def decorator(f):
        setattr(cls, f.__name__, f)
        return f 
    return decorator

@new_method(ConwayGoL)
def dummy_method(self):
    print("Note that the old value of dummy_method is overwritten!")

In [ ]:
c.dummy_method()